In [46]:
import cv2
import daria as da
import numpy as np
from scipy import ndimage, interpolate


DxDy = [284, 150]
vertical_bulge = 0.5e-6
horizontal_crop = 0.965
horizontal_bulge = -3.1e-6
horizontal_stretch = 2.35e-6
horizontal_stretch_mid = -0
vertical_shear = 2e-3
vertical_crop = 0.92

img = cv2.imread("../images/originals/NonCurvCorrected.JPG")

img = cv2.resize(img, (0, 0), fx=0.2, fy=0.2)



In [47]:
Ny, Nx, _ = img.shape

img = img.astype(np.float64)

# Centering image
x = (np.array(range(1, Nx + 1)) - round(Nx / 2)) / round(Nx / 2) * DxDy[0] / 2
y = (np.array(range(1, Ny + 1)) - round(Ny / 2)) / round(Ny / 2) * DxDy[1] / 2
y = y.reshape((1, Ny))

# Original grid
X, Y = np.meshgrid(x, y)

# Curvature corrected grid
Xmod = horizontal_crop * np.multiply(
    np.multiply(X, (1 + horizontal_stretch * (X - horizontal_stretch_mid) ** 2)),
    (1 - vertical_bulge * Y**2),
)
Ymod = vertical_crop * (
    np.multiply(Y, (1 - horizontal_bulge * X**2)) - vertical_shear * X
)

Xmod = Xmod*Nx/DxDy[0]+round(Nx/2)-1
Ymod = Ymod*Ny/DxDy[1]+round(Ny/2)-1

In [43]:
print(Ymod)
print(Y)


[[0.0528775  0.0691068  0.07814763 0.08       0.0746639  0.06213933
  0.0424263  0.01552481]
 [1.0052256  1.00348373 1.00174187 1.         0.99825813 0.99651627
  0.9947744  0.99303253]
 [1.9575737  1.93786067 1.9253361  1.92       1.92185237 1.9308932
  1.9471225  1.97054026]
 [2.90992179 2.8722376  2.84893033 2.84       2.8454466  2.86527013
  2.89947059 2.94804799]]
[[-37.5 -37.5 -37.5 -37.5 -37.5 -37.5 -37.5 -37.5]
 [  0.    0.    0.    0.    0.    0.    0.    0. ]
 [ 37.5  37.5  37.5  37.5  37.5  37.5  37.5  37.5]
 [ 75.   75.   75.   75.   75.   75.   75.   75. ]]


In [48]:
imgpre = img.astype(np.uint8)

cv2.imshow("win", imgpre)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [49]:

out_grid = np.array([Ymod.ravel(),Xmod.ravel()])
img_mod = np.zeros((Ny, Nx, 3), dtype=np.uint8)
for i in range(3):
    in_data = img[:,:,i]
    im_array = ndimage.map_coordinates(in_data, out_grid, order=1)
    img_mod[:,:,i] = im_array.reshape(img[:,:,0].shape).astype(np.uint8)
cv2.imshow("win",img_mod)
cv2.waitKey(0)
cv2.destroyAllWindows()
# cv2.imwrite("../images/modified/curvtest.jpg", im3)

In [45]:
print(im3)
print(img[:,:,0])

[[153   5   2   4   1   1   2   0]
 [  5   6   0  13  11  10  11   0]
 [ 28  21  31  17   4  17  32   0]
 [ 27  21  17  12   5   5   9   0]]
[[167.   5.   3.   4.   1.   1.   2.   7.]
 [  5.   7.   0.  13.  11.  11.  12.  20.]
 [ 30.  22.  35.  18.   3.  19.  34.  21.]
 [ 27.  22.  15.  11.   6.   3.   7.  14.]]


True

In [35]:
ip0 = interpolate.RegularGridInterpolator((y, x), img[:, :, 0])




ValueError: The points in dimension 0 must be 1-dimensional

In [25]:
ip0(Ymod, Xmod)

AttributeError: 'interp2d' object has no attribute 'grid'

In [ ]:




points = np.zeros((Ny, Nx, 2), dtype=np.float64)
points[:, :, 0] = X
points[:, :, 1] = Y

new_points = np.zeros((Ny, Nx, 2), dtype=np.float64)
new_points[:, :, 0] = Xmod
new_points[:, :, 1] = Ymod


# print(len(X.ravel()))
# print(len(Y.ravel()))
# print(len(img[:, :, 0].ravel()))


# ip1 = interpolate.interp2d(x, y, img[:, :, 1])
# ip2 = interpolate.interp2d(x, y, img[:, :, 2])

# ip0 = interpolate.interpn((X.ravel(), Y.ravel()), img[:, :, 0].ravel(), (1, 1))

# x = np.arange(0, 1, 0.02)
# y = np.arange(0, 1, 0.02)
# xx, yy = np.meshgrid(x, y)

# z = np.sin(x + y)

# ip = interpolate.interp2d(x, y, z)


# ip0 = interpolate.interp2d(
#     np.array([1, 2, 3]), np.array([1, 2, 3]), np.array([5, 6, 7])
# )

# print(ip0)


# img_mod = np.zeros((Ny, Nx, 3), dtype=np.uint8)
# img_mod[:, :, 0] = ip0(Xmod, Ymod)
# img_mod[:, :, 1] = ip1(Xmod, Ymod)
# img_mod[:, :, 2] = ip2(Xmod, Ymod)


# img_mod = interpolate.interpn(points, img, new_points)

# # for i in range(3):
# #     print(i)
# #     img_mod[:, :, i] = interpolate.griddata(
# #         points.reshape(Ny * Nx, 2), img[:, :, i].reshape(Ny * Nx, 1), (Xmod, Ymod)
# #     ).astype(np.uint8)

cv2.imwrite("images/modified/CurvCorrected.jpg", img_mod)

In [48]:
from scipy.interpolate import RegularGridInterpolator

def f(x, y):

    return 2 * x**3 + 3 * y**2

x = np.linspace(1, 4, 11)

y = np.linspace(4, 7, 22)

xg, yg = np.meshgrid(x, y, indexing='ij', sparse=True)

data = f(xg, yg)

In [52]:
my_interpolating_function = RegularGridInterpolator((x, y), data)

In [53]:
my_interpolating_function(xg,yg)

/home/erlend/src/daria/venv/lib/python3.10/site-packages/scipy/interpolate/_interpolate.py:2511: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if method not in ["linear", "nearest"]:


ValueError: Method '[[4.         4.14285714 4.28571429 4.42857143 4.57142857 4.71428571
  4.85714286 5.         5.14285714 5.28571429 5.42857143 5.57142857
  5.71428571 5.85714286 6.         6.14285714 6.28571429 6.42857143
  6.57142857 6.71428571 6.85714286 7.        ]]' is not defined